# **پروژه پایانی - پیاده سازی الگوریتم های یادگیری ماشین بر روی داده های کووید**

## گام اول: آماده سازی داده ها 

 ابتدا کتابخانه های مورد نیاز و  فایل دیتاست را لود میکنیم 

In [47]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
data = pd.read_json("/kaggle/input/covid-patient-datasets/covid.json")

In [48]:
data.info()
data.head(500)

488 <br>
سطر داده داریم که باید نرمالیزه شوند. چون سن باینری نمی شود و ستون # برای شمارنده هست باید حذف شود. و سایر داده ها به <br> 
no => 0  <br>
yes => 1 <br>
نبدیل شود


In [49]:
normalData = data.drop(['#',"age"],axis = 1)
for param in normalData:
    normalData[param] = normalData[param].apply(lambda x: 0 if x=='no' else 1)
    #normalData[param] = normalData[param].map(dict(yes=1, no=0))
normalData.head(500)

حالا باید داده های تکراری را حذف کنیم

In [50]:
normalData.drop_duplicates(inplace=True)
normalData.head(500)

از 487 داده
<br>
487 - 285 = 202 
<br>
تکراری بودند
<br>
حال تمامی این داده ها دارای برچسب 1 هستند که باید این ویژگی به داده ها اضافه شود


In [51]:
normalData["result"] = 1
normalData.head(500)

حال باید تمامی ترکیبات را بدست بیاوریم و آن هاایی که در داده ها نیستند برچسب 0 بزنیم و با داده های قبلی ترکیب کنیم

In [52]:
from itertools import product

uniques = [normalData[i].unique().tolist() for i in normalData.columns ]

# conver lable to 0
uniques[-1][0] = 0
falseData = pd.DataFrame(product(*uniques), columns = normalData.columns)


finalData = normalData.append(falseData)
finalData.head(5000000000)

الان داده هایی که تکراری که برچسب 0 دارند باید حذف بشوند

In [53]:
feature_cols = finalData.columns;
#remove the result column
feature_cols =feature_cols[:-1]

finalData.drop_duplicates(feature_cols,inplace=True)
finalData.head(5000000000)

گام اول با جمع آوری داده و نرمال کردن و ایجاد رکیب های موچود تمام شد

In [54]:
finalData.info()
finalData.describe().transpose()

# گام دوم: اجرای الگوریتم های یادگیری ماشین

## 1- بدست آوردن 5 ویژگی کم اهمیت
برای این کار از آنتروپی برای محاسیه 
<br> gain <br> 
استفاده میکنیم

In [55]:
from math import log2

# calculate the entropy 
def entropy(trueResultCount,falseResultCount):  
    p1 = trueResultCount / (trueResultCount + falseResultCount)
    p2 = falseResultCount / (trueResultCount + falseResultCount)
    return -((p1) * log2(p1) + (p2) * log2(p2))

# calculate the gain of item 
def gain(atr):
    trueResultCount = finalData[finalData[atr] == 1][atr].count() 
    falseResultCount = finalData[finalData[atr] == 0][atr].count() 
    
    trueWhenResultIsTrueCount = finalData.loc[(finalData[atr] == 1) & (finalData['result'] == 1)][atr].count()
    falseWhenResultIsTrueCount = finalData.loc[(finalData[atr] == 0) & (finalData['result'] == 1)][atr].count() 
    trueWhenResultIsFalseCount = finalData.loc[(finalData[atr] == 1) & (finalData['result'] == 0)][atr].count()
    falseWhenResultIsFalseCount = finalData.loc[(finalData[atr] == 0) & (finalData['result'] == 0)][atr].count() 
    
    return entropyS - ((trueWhenResultIsTrueCount + trueWhenResultIsFalseCount) / (trueResultCount + falseResultCount)) * entropy(trueWhenResultIsTrueCount, trueWhenResultIsFalseCount) + ((falseWhenResultIsTrueCount + falseWhenResultIsFalseCount) / (trueResultCount + falseResultCount)) * entropy(falseWhenResultIsTrueCount, falseWhenResultIsFalseCount)
  
    
 #### ------------------ ####   
    
trueResultCount = finalData[finalData.result == 1].result.count()  
falseResultCount = finalData[finalData.result == 0].result.count()

## system entropy
entropyS = round(entropy(trueResultCount, falseResultCount),8)


## list of gain attributes
gainList = []
for atr in feature_cols:
    gainResult = gain(atr)
    gainList.append([atr,round(gainResult, 8)])

همان طور که در جدول آمده است 5 ویژگی آخر از کم اهمیت ترین ویژگی ها می باشند

In [56]:
print("system entropy: ",entropyS)
df = pd.DataFrame(gainList, columns =['attribute', 'gain']) 
df = df.sort_values(by='gain', ascending=False)
df.head(50)

## 2- محاسبه الگوریتم find-S

In [57]:
#training function to implement find-s algorithm
def FindS(c,t):
    for i, val in enumerate(t):
        if val == 1:
            specific_hypothesis = c[i].copy()
            break
             
    for i, val in enumerate(c):
        if t[i] == 1:
            for x in range(len(specific_hypothesis)):
                if val[x] != specific_hypothesis[x]:
                    specific_hypothesis[x] = -1
                else:
                    pass
    
    # replace =1 with ? and return
    return  [x if x != -1 else '?' for x in specific_hypothesis] 

In [58]:
# make concept and target (array)
target = np.array(finalData.result) 
concept = np.array(finalData.drop(['result'], axis=1))


# making an array of all the attributes
d = [None] * len(finalData.columns)

h = FindS(concept,target)
print(h)

## 3- محاسبه الگوریتم CE


In [ ]:
def CE(concepts, target): 
    
    specific_h = concepts[0].copy()

    print("Initialization of specific_h and general_h")

    print("specific_h: ",specific_h)

    general_h = [["?" for i in range(len(specific_h))] for i in range(len(specific_h))]

    print("general_h: ",general_h)

    print("concepts: ",concepts)

    for i, h in enumerate(concepts):

        if target[i] == "yes":

            for x in range(len(specific_h)):

                #print("h[x]",h[x])

                if h[x] != specific_h[x]:

                    specific_h[x] = '?'

                    general_h[x][x] = '?'

        if target[i] == "no":

            for x in range(len(specific_h)):

                if h[x] != specific_h[x]:

                    general_h[x][x] = specific_h[x]

                else:

                    general_h[x][x] = '?'

    print("\nSteps of Candidate Elimination Algorithm: ",i+1)

    print("Specific_h: ",i+1)

    print(specific_h,"\n")

    print("general_h :", i+1)

    print(general_h)

    indices = [i for i, val in enumerate(general_h) if val == ['?', '?', '?', '?', '?', '?']]

    print("\nIndices",indices)

    for i in indices:

        general_h.remove(['?', '?', '?', '?', '?', '?'])

    return specific_h, general_h







s_final, g_final = CE(concept,target)

print("Final Specific_h: ", s_final, sep="\n")
print("Final General_h: ", g_final, sep="\n")

### ساحت داده تست 

In [ ]:
X = finalData.drop(['result'],axis = 1)
y = finalData.result.values

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)


# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## 4-  محاسبه الگوریتم بیز

In [ ]:
import matplotlib.pyplot as plt



# Training the Naive Bayes model on the Training set
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score
ac = accuracy_score(y_test,y_pred)
cm = confusion_matrix(y_test, y_pred)

print("NB accuracy score: ", ac )

## 5-  محاسبه الگوریتم knn


به دلیل زمان بر بودن کامنت شده است 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=3)
#classifier.fit(X_train, y_train)

##Predict the response for test dataset
#y_pred = classifier.predict(X_test)

##Import scikit-learn metrics module for accuracy calculation
#from sklearn import metrics
## Model Accuracy, how often is the classifier correct?
#print("Knn Accuracy: ",metrics.accuracy_score(y_test, y_pred))

## 6-  محاسبه الگوریتم کلاسترینگ

چون ویژگی های ما بیشتر از 2 بعد هست نمی شود آن ها را روی نمودار نمایش داد 


In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3).fit(finalData)
centroids = kmeans.cluster_centers_
print(centroids)

# plt.scatter(finalData['x'], finalData['y'], c= kmeans.labels_.astype(float), s=50, alpha=0.5)
# plt.scatter(centroids[:, 0], centroids[:, 1], c='red', s=50)
# plt.show()

## 7-  محاسبه الگوریتم درخت تصمیم تصادفی

In [ ]:
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

# Create Decision Tree classifer object
clf = DecisionTreeClassifier()
# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)

# Model Accuracy, how often is the classifier correct?
print("DecisionTree accuracy: ",metrics.accuracy_score(y_test, y_pred))

## 8-  محاسبه الگوریتم درخت تصمیم ID3

In [ ]:
import math
from collections import deque

class Node:
    """Contains the information of the node and another nodes of the Decision Tree."""

    def __init__(self):
        self.value = None
        self.next = None
        self.childs = None


class DecisionTreeClassifier:
    """Decision Tree Classifier using ID3 algorithm."""

    def __init__(self, X, feature_names, labels):
        self.X = X
        self.feature_names = feature_names
        self.labels = labels
        self.labelCategories = list(set(labels))
        self.labelCategoriesCount = [list(labels).count(x) for x in self.labelCategories]
        self.node = None
        self.entropy = self._get_entropy([x for x in range(len(self.labels))])  # calculates the initial entropy

    def _get_entropy(self, x_ids):
        """ Calculates the entropy.
        Parameters
        __________
        :param x_ids: list, List containing the instances ID's
        __________
        :return: entropy: float, Entropy.
        """
        # sorted labels by instance id
        labels = [self.labels[i] for i in x_ids]
        # count number of instances of each category
        label_count = [labels.count(x) for x in self.labelCategories]
        # calculate the entropy for each category and sum them
        entropy = sum([-count / len(x_ids) * math.log(count / len(x_ids), 2) if count else 0 for count in label_count])
        return entropy

    def _get_information_gain(self, x_ids, feature_id):
        """Calculates the information gain for a given feature based on its entropy and the total entropy of the system.
        Parameters
        __________
        :param x_ids: list, List containing the instances ID's
        :param feature_id: int, feature ID
        __________
        :return: info_gain: float, the information gain for a given feature.
        """
        # calculate total entropy
        info_gain = self._get_entropy(x_ids)
        # store in a list all the values of the chosen feature
        x_features = [self.X[x][feature_id] for x in x_ids]
        # get unique values
        feature_vals = list(set(x_features))
        # get frequency of each value
        feature_vals_count = [x_features.count(x) for x in feature_vals]
        # get the feature values ids
        feature_vals_id = [
            [x_ids[i]
            for i, x in enumerate(x_features)
            if x == y]
            for y in feature_vals
        ]

        # compute the information gain with the chosen feature
        info_gain = info_gain - sum([val_counts / len(x_ids) * self._get_entropy(val_ids)
                                     for val_counts, val_ids in zip(feature_vals_count, feature_vals_id)])

        return info_gain

    def _get_feature_max_information_gain(self, x_ids, feature_ids):
        """Finds the attribute/feature that maximizes the information gain.
        Parameters
        __________
        :param x_ids: list, List containing the samples ID's
        :param feature_ids: list, List containing the feature ID's
        __________
        :returns: string and int, feature and feature id of the feature that maximizes the information gain
        """
        # get the entropy for each feature
        features_entropy = [self._get_information_gain(x_ids, feature_id) for feature_id in feature_ids]
        # find the feature that maximises the information gain
        max_id = feature_ids[features_entropy.index(max(features_entropy))]

        return self.feature_names[max_id], max_id

    def id3(self):
        """Initializes ID3 algorithm to build a Decision Tree Classifier.
        :return: None
        """
        x_ids = [x for x in range(len(self.X))]
        feature_ids = [x for x in range(len(self.feature_names))]
        self.node = self._id3_recv(x_ids, feature_ids, self.node)
        print('')

    def _id3_recv(self, x_ids, feature_ids, node):
        """ID3 algorithm. It is called recursively until some criteria is met.
        Parameters
        __________
        :param x_ids: list, list containing the samples ID's
        :param feature_ids: list, List containing the feature ID's
        :param node: object, An instance of the class Nodes
        __________
        :returns: An instance of the class Node containing all the information of the nodes in the Decision Tree
        """
        if not node:
            node = Node()  # initialize nodes
        # sorted labels by instance id
        labels_in_features = [self.labels[x] for x in x_ids]
        # if all the example have the same class (pure node), return node
        if len(set(labels_in_features)) == 1:
            node.value = self.labels[x_ids[0]]
            return node
        # if there are not more feature to compute, return node with the most probable class
        if len(feature_ids) == 0:
            node.value = max(set(labels_in_features), key=labels_in_features.count)  # compute mode
            return node
        # else...
        # choose the feature that maximizes the information gain
        best_feature_name, best_feature_id = self._get_feature_max_information_gain(x_ids, feature_ids)
        node.value = best_feature_name
        node.childs = []
        # value of the chosen feature for each instance
        feature_values = list(set([self.X[x][best_feature_id] for x in x_ids]))
        # loop through all the values
        for value in feature_values:
            child = Node()
            child.value = value  # add a branch from the node to each feature value in our feature
            node.childs.append(child)  # append new child node to current node
            child_x_ids = [x for x in x_ids if self.X[x][best_feature_id] == value]
            if not child_x_ids:
                child.next = max(set(labels_in_features), key=labels_in_features.count)
                print('')
            else:
                if feature_ids and best_feature_id in feature_ids:
                    to_remove = feature_ids.index(best_feature_id)
                    feature_ids.pop(to_remove)
                # recursively call the algorithm
                child.next = self._id3_recv(child_x_ids, feature_ids, child.next)
        return node

    def printTree(self):
        if not self.node:
            return
        nodes = deque()
        nodes.append(self.node)
        while len(nodes) > 0:
            node = nodes.popleft()
            print(node.value)
            if node.childs:
                for child in node.childs:
                    print('({})'.format(child.value))
                    nodes.append(child.next)
            elif node.next:
                print(node.next)

In [ ]:
tree = DecisionTreeClassifier(X=concept, feature_names=finalData.columns, labels=target)
print("System entropy {:.8f}".format(tree.entropy))
tree.printTree()